### Training process

- Design model (input size, output size, forward pass)
- Construct loss and optimizer
- Training loop
  - forward pass: compute prediction
  - backward pass: gradients
  - update weights


### Import package


In [6]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### Prepared data


In [7]:
bc = datasets.load_breast_cancer()
x, y = bc.data, bc.target

n_samples, n_features = x.shape

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=123)

# scale
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

# numpy->torch
x_train = torch.from_numpy(x_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# reshape
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

### Model


In [8]:
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred


model = LogisticRegression(n_features)

### Loss and optimizer


In [9]:
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Training loop


In [10]:
num_epochs = 100
for epoch in range(num_epochs):
    # forward pass and loss
    y_pred = model(x_train)
    loss = criterion(y_pred, y_train)

    # backward
    loss.backward()

    # update
    optimizer.step()

    # zero gradients
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss: {loss.item():.4f}')

with torch.no_grad():
    y_pred = model(x_test)
    y_pred_cls = y_pred.round()
    acc = y_pred_cls.eq(y_test).sum()/float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

epoch: 10, loss: 0.3940
epoch: 20, loss: 0.3531
epoch: 30, loss: 0.3231
epoch: 40, loss: 0.3000
epoch: 50, loss: 0.2816
epoch: 60, loss: 0.2665
epoch: 70, loss: 0.2538
epoch: 80, loss: 0.2430
epoch: 90, loss: 0.2336
epoch: 100, loss: 0.2254
accuracy = 0.9737
